In [1]:
import os
from bs4 import BeautifulSoup
import math
import numpy as np
import shapely as shp
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.mask

### Set the working directory (absolute)

In [2]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

### Set the site name and its latitude and longitude

In [3]:
# Site Name
site_Name = "Tapajos KM67 Mature Forest site"
site_Name_New = "Tapajos KM67 Mature Forest site"

In [4]:
# Find the coordinate based on the name of the site in our .csv file
df_Site = pd.read_csv(cwd + "//Site.csv")
for i in range(df_Site.shape[0]):
    if df_Site.loc[i,"Site"] == site_Name_New:
        site_Lat = df_Site.loc[i,"Latitude"]
        site_Lon = df_Site.loc[i,"Longitude"]
# print(f"The latitude of the site is {site_Lat:.4f} and the longitude is {site_Lon:.4f}")

### Create a geodataframe of the site

In [5]:
df_4326 = pd.DataFrame({
    "Site": [site_Name_New],
    "Latitude": [site_Lat],
    "Longitude": [site_Lon]
})
df_4326

,Site,Latitude,Longitude
0,Tapajos KM67 Mature Forest site,-2.8567,-54.9589


In [6]:
# Create a point shapefile based on the site, using Lon-Lat
gdf_4326 = gpd.GeoDataFrame(
    df_4326,
    geometry = gpd.points_from_xy(df_4326['Longitude'], df_4326['Latitude']),
    crs = "EPSG:4326"
)
gdf_4326

,Site,Latitude,Longitude,geometry
0,Tapajos KM67 Mature Forest site,-2.8567,-54.9589,POINT (-54.9589 -2.8567)


### Get the paths to the raw L1C rasters
We only need the B08 (NIR) image file. 

In [7]:
for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
    for name in files:
        temp = os.path.join(path, name)
        if "IMG_DATA" in temp and temp[-3:] == 'jp2' and "B08" in temp:
            # print(temp)
            path_L1C_B08_raw = temp
path_L1C_B08_raw

'c:\\Users\\m1865\\Desktop\\DISC\\Sentinel-2 Images Raw\\Tapajos KM67 Mature Forest site\\L1C\\S2B_MSIL1C_20230819T135709_N0509_R067_T21MYS_20230819T172029.SAFE\\GRANULE\\L1C_T21MYS_A033698_20230819T135707\\IMG_DATA\\T21MYS_20230819T135709_B08.jp2'

### Get the path to the XML file "MTD_DS" of L1C raster, where there are values of "U", "Solar Irradiance", "Quantification Value" and "Radiometric Offset"

In [8]:
for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
    for name in files:
        temp = os.path.join(path, name)
        if "MTD_DS.xml" in temp:
            # print(temp)
            path_L1C_xml_DS = temp
path_L1C_xml_DS

'c:\\Users\\m1865\\Desktop\\DISC\\Sentinel-2 Images Raw\\Tapajos KM67 Mature Forest site\\L1C\\S2B_MSIL1C_20230819T135709_N0509_R067_T21MYS_20230819T172029.SAFE\\DATASTRIP\\DS_2BPS_20230819T172029_S20230819T135707\\MTD_DS.xml'

### Get the path to the XML file "MTD_TL" of L1C raster, where there are values (matrices) of "Solar Zenith Angle". 

In [9]:
for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
    for name in files:
        temp = os.path.join(path, name)
        if "MTD_TL.xml" in temp:
            # print(temp)
            path_L1C_xml_TL = temp
path_L1C_xml_TL

'c:\\Users\\m1865\\Desktop\\DISC\\Sentinel-2 Images Raw\\Tapajos KM67 Mature Forest site\\L1C\\S2B_MSIL1C_20230819T135709_N0509_R067_T21MYS_20230819T172029.SAFE\\GRANULE\\L1C_T21MYS_A033698_20230819T135707\\MTD_TL.xml'

### Get the paths to the raw L2A rasters
We get the paths to the B04 (Red) band and B08 (NIR) band. 

In [10]:
for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L2A"):
    for name in files:
        temp = os.path.join(path, name)
        if temp[-3:] == 'jp2'in temp and "10m" in temp and "B04" in temp :
            path_L2A_B04_raw = temp
        if temp[-3:] == 'jp2'in temp and "10m" in temp and "B08" in temp :
            path_L2A_B08_raw = temp
print("The path to B04 is " + path_L2A_B04_raw)
print("The path to B08 is " + path_L2A_B08_raw)

The path to B04 is c:\Users\m1865\Desktop\DISC\Sentinel-2 Images Raw\Tapajos KM67 Mature Forest site\L2A\S2B_MSIL2A_20230819T135709_N0509_R067_T21MYS_20230819T181035.SAFE\GRANULE\L2A_T21MYS_A033698_20230819T135707\IMG_DATA\R10m\T21MYS_20230819T135709_B04_10m.jp2
The path to B08 is c:\Users\m1865\Desktop\DISC\Sentinel-2 Images Raw\Tapajos KM67 Mature Forest site\L2A\S2B_MSIL2A_20230819T135709_N0509_R067_T21MYS_20230819T181035.SAFE\GRANULE\L2A_T21MYS_A033698_20230819T135707\IMG_DATA\R10m\T21MYS_20230819T135709_B08_10m.jp2


### Get the path to the XML file "MTD_DS" of L1C raster, where there are values of "Quantification Value" and "Radiometric Offset"

In [11]:
for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L2A"):
    for name in files:
        temp = os.path.join(path, name)
        if "MTD_DS.xml" in temp:
            # print(temp)
            path_L2A_xml_DS = temp
path_L2A_xml_DS

'c:\\Users\\m1865\\Desktop\\DISC\\Sentinel-2 Images Raw\\Tapajos KM67 Mature Forest site\\L2A\\S2B_MSIL2A_20230819T135709_N0509_R067_T21MYS_20230819T181035.SAFE\\DATASTRIP\\DS_2BPS_20230819T181035_S20230819T135707\\MTD_DS.xml'

### Now read the path into image with rasterio

In [12]:
image_L1C_B08 = rio.open(path_L1C_B08_raw)
image_L2A_B04 = rio.open(path_L2A_B04_raw)
image_L2A_B08 = rio.open(path_L2A_B08_raw)
type(image_L2A_B08)

rasterio.io.DatasetReader

### And then read the values of each image

In [13]:
values_L1C_B08 = image_L1C_B08.read(1)
values_L2A_B04 = image_L2A_B04.read(1)
values_L2A_B08 = image_L2A_B08.read(1)

### Now check the crs of the Sentinel-2 images, and convert our site geodataframe to that crs

In [14]:
crs_L1C = image_L1C_B08.crs.data["init"].split(":")[1]
crs_L2A = image_L2A_B04.crs.data["init"].split(":")[1]
print("The EPSG of L1C is " + crs_L1C)
print("The EPSG of L2A is " + crs_L2A)
# In the case that L1C and L2A have different crs, give an error. But this shouldn't happen. 
if crs_L2A != crs_L1C:
    raise SystemExit("Stop right there!")
crs_Final = 'EPSG:' + crs_L1C
print("The final crs is " + crs_Final)

The EPSG of L1C is 32721
The EPSG of L2A is 32721
The final crs is EPSG:32721


In [15]:
# Converting Lon-Lat to UTM coordinates!
# Attention that the CRS in USA for meter unit is 32618!!! Not 32632!!! 
gdf_New = gdf_4326.copy()
gdf_New = gdf_New.to_crs(crs_Final)
gdf_New

,Site,Latitude,Longitude,geometry
0,Tapajos KM67 Mature Forest site,-2.8567,-54.9589,POINT (726891.047 9684044.712)


### Now we assign the site location to a pixel in the image

In [16]:
# First we retrieve the x, y coordinate of our site
site_x = gdf_New.geometry.x.values[0]
site_y = gdf_New.geometry.y.values[0]
print("Site X, Y: " + str(site_x) + "," + str(site_y))

Site X, Y: 726891.0466288019,9684044.711766005


In [17]:
# Then we get the row, col of our site in the raster, so that we know the pixel the site belongs to
site_row, site_col = image_L2A_B08.index(site_x, site_y)
print("Site falls on the pixel: " + str(site_row) + ", " + str(site_col))

Site falls on the pixel: 1595, 2693


In [18]:
# Now we get the spatial coordinates of that pixel (coordinates of its center)
site_pixel_x, site_pixel_y = image_L2A_B08.xy(site_row, site_col)
print("The spatial coordinates of the (center) pixel which the site falls on is: " + str(site_pixel_x) + ", " + str(site_pixel_y))

The spatial coordinates of the (center) pixel which the site falls on is: 726895.0, 9684045.0


### Now we create four squares centered at our site pixel! 

In [19]:
# Get the coordinates of the four corners
# 10m
site_x_left_10m = site_pixel_x - 5
site_x_right_10m = site_pixel_x + 5
site_y_top_10m = site_pixel_y + 5
site_y_bottom_10m = site_pixel_y - 5
# 30m
site_x_left_30m = site_pixel_x - 15
site_x_right_30m = site_pixel_x + 15
site_y_top_30m = site_pixel_y + 15
site_y_bottom_30m = site_pixel_y - 15
# 100m
site_x_left_100m = site_pixel_x - 55
site_x_right_100m = site_pixel_x + 55
site_y_top_100m = site_pixel_y + 55
site_y_bottom_100m = site_pixel_y - 55
# 100 * 1.5 = 150m
site_x_left_150m = site_pixel_x - 75
site_x_right_150m = site_pixel_x + 75
site_y_top_150m = site_pixel_y + 75
site_y_bottom_150m = site_pixel_y - 75
# 300m
site_x_left_300m = site_pixel_x - 155
site_x_right_300m = site_pixel_x + 155
site_y_top_300m = site_pixel_y + 155
site_y_bottom_300m = site_pixel_y - 155
# 300 * 1.5 = 450m
site_x_left_450m = site_pixel_x - 225
site_x_right_450m = site_pixel_x + 225
site_y_top_450m = site_pixel_y + 225
site_y_bottom_450m = site_pixel_y - 225
# 600m
site_x_left_600m = site_pixel_x - 305
site_x_right_600m = site_pixel_x + 305
site_y_top_600m = site_pixel_y + 305
site_y_bottom_600m = site_pixel_y - 305
# 900m
site_x_left_900m = site_pixel_x - 455
site_x_right_900m = site_pixel_x + 455
site_y_top_900m = site_pixel_y + 455
site_y_bottom_900m = site_pixel_y - 455
# 900 * 1.5 = 1350m
site_x_left_1350m = site_pixel_x - 675
site_x_right_1350m = site_pixel_x + 675
site_y_top_1350m = site_pixel_y + 675
site_y_bottom_1350m = site_pixel_y - 675
# 1200m
site_x_left_1200m = site_pixel_x - 605
site_x_right_1200m = site_pixel_x + 605
site_y_top_1200m = site_pixel_y + 605
site_y_bottom_1200m = site_pixel_y - 605
# 1800m
site_x_left_1800m = site_pixel_x - 905
site_x_right_1800m = site_pixel_x + 905
site_y_top_1800m = site_pixel_y + 905
site_y_bottom_1800m = site_pixel_y - 905
# 2500m
site_x_left_2500m = site_pixel_x - 1255
site_x_right_2500m = site_pixel_x + 1255
site_y_top_2500m = site_pixel_y + 1255
site_y_bottom_2500m = site_pixel_y - 1255

In [20]:
# Now we need to form squares shapefile, which will be the internal area of which we will evaluate the spatial representativeness. 
shp_10m = shp.box(site_x_left_10m, site_y_bottom_10m, site_x_right_10m, site_y_top_10m)
gdf_10m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_10m],
    crs = crs_Final
)
shp_30m = shp.box(site_x_left_30m, site_y_bottom_30m, site_x_right_30m, site_y_top_30m)
gdf_30m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_30m],
    crs = crs_Final
)
shp_100m = shp.box(site_x_left_100m, site_y_bottom_100m, site_x_right_100m, site_y_top_100m)
gdf_100m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_100m],
    crs = crs_Final
)
shp_150m = shp.box(site_x_left_150m, site_y_bottom_150m, site_x_right_150m, site_y_top_150m)
gdf_150m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_150m],
    crs = crs_Final
)
shp_300m = shp.box(site_x_left_300m, site_y_bottom_300m, site_x_right_300m, site_y_top_300m)
gdf_300m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_300m],
    crs = crs_Final
)
shp_450m = shp.box(site_x_left_450m, site_y_bottom_450m, site_x_right_450m, site_y_top_450m)
gdf_450m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_450m],
    crs = crs_Final
)
shp_600m = shp.box(site_x_left_600m, site_y_bottom_600m, site_x_right_600m, site_y_top_600m)
gdf_600m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_600m],
    crs = crs_Final
)
shp_900m = shp.box(site_x_left_900m, site_y_bottom_900m, site_x_right_900m, site_y_top_900m)
gdf_900m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_900m],
    crs = crs_Final
)
shp_1350m = shp.box(site_x_left_1350m, site_y_bottom_1350m, site_x_right_1350m, site_y_top_1350m)
gdf_1350m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_1350m],
    crs = crs_Final
)
shp_1200m = shp.box(site_x_left_1200m, site_y_bottom_1200m, site_x_right_1200m, site_y_top_1200m)
gdf_1200m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_1200m],
    crs = crs_Final
)
shp_1800m = shp.box(site_x_left_1800m, site_y_bottom_1800m, site_x_right_1800m, site_y_top_1800m)
gdf_1800m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_1800m],
    crs = crs_Final
)
shp_2500m = shp.box(site_x_left_2500m, site_y_bottom_2500m, site_x_right_2500m, site_y_top_2500m)
gdf_2500m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_2500m],
    crs = crs_Final
)

### Calculate NDVI based on the L2A image! This calculation is performed on the entire image, and the image cropping will be performed later. 

In [27]:
# Read the DS xml file of L2A
with open(path_L2A_xml_DS, 'r') as f:
    data = f.read()
BS_L2A_dS = BeautifulSoup(data, "xml")

In [35]:
# Get the quantification value! 
quantification_L2A = int(BS_L2A_dS.find("BOA_QUANTIFICATION_VALUE").text)
# Get the radiometric offset!
offset_L2A_B04 = abs(int(BS_L2A_dS.find("BOA_ADD_OFFSET", {"band_id": "3"}).text))
offset_L2A_B08 = abs(int(BS_L2A_dS.find("BOA_ADD_OFFSET", {"band_id": "7"}).text))
offset_L2A_B08

1000

In [33]:
# Calculate NDVI of L2A! 
NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )
NDVI

array([[ 0.46726047,  0.46914021,  0.47102313, ...,  0.40375123,
         0.45275035,  0.49636225],
       [ 0.46319393,  0.46362649,  0.4695392 , ...,  0.46406268,
         0.44605065,  0.41604456],
       [ 0.45807089,  0.47063789,  0.47569816, ...,  0.41875761,
         0.3439582 ,  0.30219992],
       ...,
       [-0.0051619 , -0.00775558, -0.00634548, ...,  0.15353371,
         0.14285714,  0.11974954],
       [-0.005145  , -0.01148886, -0.0112782 , ...,  0.14784876,
         0.11643657,  0.11859059],
       [-0.00611189, -0.01129412, -0.00636042, ...,  0.12797927,
         0.11227978,  0.11081864]])

In [34]:
src = image_L2A_B04
out_meta = src.meta
out_meta.update({
    "driver": "GTiff",
    "dtype": 'float64'
})
with rio.open(cwd_Images_Processed + "\\" + site_Name_New + "\\NDVI No Offset.tif", 'w', **out_meta) as dest:
    dest.write(NDVI, 1)

### Parse XML file of L1C images! We start with MTD_DS.xml which is easier to parse. 

In [37]:
# Read the DS xml file of L1C
with open(path_L1C_xml_DS, 'r') as f:
    data = f.read()
BS_L1C_dS = BeautifulSoup(data, "xml")

In [38]:
# Get the quantification value! 
quantification_L1C = int(BS_L1C_dS.find("QUANTIFICATION_VALUE").text)
# Get the radiometric offset!
offset_L1C = int(BS_L1C_dS.find("RADIO_ADD_OFFSET", {"band_id": "7"}).text)
# Get the U
U_L1C = float(BS_L1C_dS.find("U").text)
# Get the solar irradiance
SolarIrr = float(BS_L1C_dS.find("SOLAR_IRRADIANCE", {"bandId": "7"}).text)
quantification_L1C

10000

### Now we parse MTD_TL.xml to get the sun zenith angle! 

In [39]:
# Read the TL xml file of L1C
with open(path_L1C_xml_TL, 'r') as f:
    data = f.read()
BS_L1C_dS = BeautifulSoup(data, "xml")

In [40]:
# Get the sun zenith angle! There should be a 23 x 23 arrays in the xml. Now we save each row as an array and keep all these arrays into a list
list_SunZenith = []
for row in BS_L1C_dS.find("Sun_Angles_Grid").find("Zenith").find_all("VALUES"):
    temp_List = row.text.split(" ")
    temp_Arr = np.array(temp_List)
    temp_Arr = temp_Arr.astype(float)
    list_SunZenith.append(temp_Arr)
# Now we stack these nested-in-list arrays into a 2d array
index = 0
for arr in list_SunZenith:
    if index == 0:
        arr_SunZenith = arr
    else:
        arr_SunZenith = np.vstack((arr_SunZenith, arr))
    index = index + 1
arr_SunZenith.shape

(23, 23)

### Now since that we have retrieved the sun zenith angle, we have to assign that value to each pixel, due to the different spatial resolution. 

In [41]:
# Get the shape of L1C image, which should be (10980, 10980)
shape_L1C = values_L1C_B08.shape
shape_L1C

(10980, 10980)

In [42]:
# Repeat each element of sun zenith angle array, in both axies. The final array should have a shape of (11500, 11500)
arr_SunZenith_Repeat = np.repeat(arr_SunZenith, 500, axis = 1)
arr_SunZenith_Repeat = np.repeat(arr_SunZenith_Repeat, 500, axis = 0)
arr_SunZenith_Repeat.shape

(11500, 11500)

In [43]:
# Index only the first 10980 of each dimension
arr_SunZenith_Assigned = arr_SunZenith_Repeat[0:shape_L1C[0], 0:shape_L1C[1]]
arr_SunZenith_Assigned.shape

(10980, 10980)

### Now, finally we can calculate the radiance of L1C from reflectance

In [44]:
offset_L1C

-1000

In [246]:
# radiance = reflectance * cos(radians(SunZenithAngle)) * solarIrradiance * U / pi
radiance = (values_L1C_B08 + offset_L1C).astype(float)  * np.cos(np.radians(arr_SunZenith_Assigned)) * SolarIrr / quantification_L1C / (math.pi * (1 / U_L1C))
radiance 

array([[101.83195164, 105.34169366, 108.30767283, ...,  76.25307626,
         79.48098245,  77.79253921],
       [111.27365201, 115.37658986, 116.71128049, ...,  84.09937131,
         79.92792331,  82.93235907],
       [118.14483709, 121.72872859, 121.13553276, ...,  88.07217893,
         85.16706336,  83.6772605 ],
       ...,
       [ 77.76778749,  72.93391213,  60.13541313, ..., 154.32550846,
        165.24452879, 170.55308475],
       [ 63.59175924,  62.41459788,  47.23673018, ..., 161.57130997,
        171.08142445, 173.37089645],
       [ 65.79580603,  46.61058052,  32.3844603 , ..., 177.82404529,
        184.18928064, 185.49755036]])

### Now we finally calculate the NDVI * Rad

In [247]:
NIRv = NDVI * radiance
NIRv

array([[ 19.89549113,  17.86601263,  19.52708846, ...,  58.65621251,
         61.22777816,  57.71984828],
       [ 21.92070068,  19.56917425,  20.35542929, ...,  70.02182106,
         63.31177721,  65.27897135],
       [ 19.18223429,  18.24681147,  18.9415578 , ...,  76.77301326,
         71.08057799,  67.10920584],
       ...,
       [ 69.45796396,  57.86632997,  30.17130587, ..., -11.8482899 ,
        -12.88389139, -11.21753741],
       [ 56.11274108,  39.02194859,  13.82203332, ..., -13.05299119,
        -12.96393403, -12.30798983],
       [ 57.83503095,  30.6217603 ,   9.87611201, ..., -11.94028763,
        -12.52618869, -14.02657772]])

In [248]:
src = image_L1C_B08
out_meta = src.meta
out_meta.update({
    "driver": "GTiff",
    "dtype": 'float64'
})
with rio.open(cwd_Images_Processed + "\\" + site_Name_New + "\\NIRv.tif", 'w', **out_meta) as dest:
    dest.write(NIRv, 1)

### Cropping! 

In [249]:
list_Distance = ['10m','30m','100m','150m','300m','450m','600m','900m','1350m','1200m','1800m','2500m']
list_GDF = [gdf_10m,gdf_30m,gdf_100m,gdf_150m,gdf_300m,gdf_450m,gdf_600m,gdf_900m,gdf_1350m,gdf_1200m,gdf_1800m,gdf_2500m]

In [250]:
for index in range(len(list_Distance)):
    temp_Distance = list_Distance[index]
    temp_GDF = list_GDF[index]
    src = rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv.tif")
    out_image, out_transform = rio.mask.mask(src, temp_GDF.geometry, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})

    with rio.open(cwd_Images_Processed + "\\" + site_Name_New + "\\NIRv " + temp_Distance + ".tif", "w", **out_meta) as dest:
        dest.write(out_image)

### Bonus: Export those shapefile to local storage for future visualization

In [251]:
list_Distance = ['30m','100m','150m','300m','450m','600m','900m','1350m','1200m','1800m','2500m']
list_GDF = [gdf_30m,gdf_100m,gdf_150m,gdf_300m,gdf_450m,gdf_600m,gdf_900m,gdf_1350m,gdf_1200m,gdf_1800m,gdf_2500m]

In [252]:
for i in range(len(list_Distance)):
    list_GDF[i].to_file(cwd_Images_Processed + "\\" + site_Name_New + "\\" + list_Distance[i] + ".shp")

In [253]:
# ### RESAMPLE
# from rasterio.enums import Resampling

# # Resample 1800m (10m pixel -> 20m pixel)

# scale_factor = 0.5

# with rasterio.open(cwd_Images_Processed + "\\" + site_Name + "\\Nirv 1800m.tif") as dataset:

#     # resample data to target shape
#     data = dataset.read(
#         out_shape=(
#             dataset.count,
#             int(dataset.height * scale_factor),
#             int(dataset.width * scale_factor)
#         ),
#         resampling=Resampling.lanczos
#     )

#     # scale image transform
#     transform = dataset.transform * dataset.transform.scale(
#         (dataset.width / data.shape[-1]),
#         (dataset.height / data.shape[-2])
#     )

#     out_meta = dataset.meta
#     out_meta.update({
#         "height": data.shape[-2],
#         "width": data.shape[-1],
#         "transform": transform
#         })

#     with rio.open(cwd_Images_Processed + "\\" + site_Name + "\\Nirv 1800m RE.tif", "w", **out_meta) as dest:
#         dest.write(out_image)

In [21]:
gdf_600m.to_file(cwd_Images_Processed + "\\" + site_Name + "\\600m.shp")

In [255]:
# gdf_4326.to_file(cwd_Images_Processed + "\\" + site_Name + "\\" + "site.shp")